In [30]:
import pandas as pd
import re
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/carlosruiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/carlosruiz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/carlosruiz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
df2 = pd.read_csv('vault_reviews.csv')
df2

,company,company_rating,title,uppers,downers,advice,date,author
0,Wipro Limited\nRelated Internships: Wipro Glo...,3.5,“Need to build a pro-active culture as per Glo...,Work life balance\nCampus facilities\nWork fro...,\nCulture\nPro-activeness\nOnboarding experien...,\nThe culture is very laid-back and employees ...,March 2020,FORMER EMPLOYEE
1,Wipro Limited\nRelated Internships: Wipro Glo...,3.5,“Great learning opportunity with challenges th...,Great work culture\n,\nNone\n,\nGreat work culture and have the passion to d...,"Experienced in Energy & UtilitiesLas Vegas, NV...",VERIFIED EMPLOYEE
2,Wipro Limited\nRelated Internships: Wipro Glo...,3.5,"“Excellent work culture, great leadership”",Partner Ecosystem\n,\nNo Comments\n,\nNo Comments,"Executive in Retail & Consumer GoodsNew York, ...",VERIFIED EMPLOYEE
3,Wipro Limited\nRelated Internships: Wipro Glo...,3.5,“Wipro has many good people with a strong inno...,"People, autonomy, entrepreneurial spirit\n","\nBack office processes, gender bias\n",\nNo large technology services and consulting ...,"Partner in Energy & UtilitiesHouston, TX2019",VERIFIED EMPLOYEE
4,Wipro Limited\nRelated Internships: Wipro Glo...,3.5,“Opportunities abound - take the bull by the h...,Flexibility to be self driving. I can set my o...,\nIndian National companies have signifcant ba...,\nIf you are self driven and want a chance at ...,"Partner in StrategyCharlotte, NC2019",VERIFIED EMPLOYEE
...,...,...,...,...,...,...,...,...
10918,RSM US LLP\n,4.0,“Experienced in Audit & Assurance”,"Within the middle market, there are many oppor...",\nLack of communication and engagment from par...,\nBusiness outlook is strong and one would ass...,"Raleigh, NC2013",VERIFIED EMPLOYEE
10919,RSM US LLP\n,4.0,“Experienced in Audit”,"New opportunities, large firm, good technology...",\npay and benefits stink.\n,\nn/a,"Chicago, IL2012",VERIFIED EMPLOYEE
10920,RSM US LLP\n,4.0,“Executive in Tax”,Some nice people\n,\nSurprisingly very political culture\n,\nYou can be successful here just not as succe...,"Minneapolis, MN2012",VERIFIED EMPLOYEE
10921,RSM US LLP\n,4.0,“Experienced in Audit”,going home,NaN,NaN,"New York, NY2010",VERIFIED EMPLOYEE


In [20]:
df = pd.read_csv('Vault_diversity_reviews - Vault_diversity_reviews.csv',index_col='Unnamed: 0')
df

,company,company_rating,title,content,author,year
0,Kenway Consulting,3.5,diversity_review,As someone who has visibility into the financi...,Executive in Data & AnalyticsChicago,['2020']
1,Kenway Consulting,3.5,diversity_review,We have a guiding principle to avoid collectiv...,Partner in Data & AnalyticsScottsdale,['2020']
2,Kenway Consulting,3.5,diversity_review,"We hire based on skill set, your potential to ...",Executive in Data & AnalyticsChicago,['2020']
3,Kenway Consulting,3.5,diversity_review,Our CEO has an equal representation of direct ...,Executive in Business OperationsChicago,['2020']
4,Kenway Consulting,3.5,diversity_review,Diversity for women is effective. Diversity fo...,Executive in CybersecurityChicago,['2020']
...,...,...,...,...,...,...
1212,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,"['The firm accepts people of all race, gender ...",Mid-levelPalo Alto,['2018']
1213,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,['Best: The gender ratio is fairly 1:1 and eve...,San Jose,['2018']
1214,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,"['new class of ~50+, noticeably male-dominant'...",Palo Alto,['2018']
1215,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,['We have very serious issues with diversity a...,Palo Alto,['2018']


In [21]:
def clean_text(text):
    '''
    Uses regex for removing punctuation and special characters.
    '''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub(r'^?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)

    return text

re_clean = lambda x: clean_text(x)

In [23]:
df['clean_review'] = pd.DataFrame(df.content.apply(re_clean))
df

,company,company_rating,title,content,author,year,clean_review
0,Kenway Consulting,3.5,diversity_review,As someone who has visibility into the financi...,Executive in Data & AnalyticsChicago,['2020'],as someone who has visibility into the financi...
1,Kenway Consulting,3.5,diversity_review,We have a guiding principle to avoid collectiv...,Partner in Data & AnalyticsScottsdale,['2020'],we have a guiding principle to avoid collectiv...
2,Kenway Consulting,3.5,diversity_review,"We hire based on skill set, your potential to ...",Executive in Data & AnalyticsChicago,['2020'],we hire based on skill set your potential to ...
3,Kenway Consulting,3.5,diversity_review,Our CEO has an equal representation of direct ...,Executive in Business OperationsChicago,['2020'],our ceo has an equal representation of direct ...
4,Kenway Consulting,3.5,diversity_review,Diversity for women is effective. Diversity fo...,Executive in CybersecurityChicago,['2020'],diversity for women is effective diversity fo...
...,...,...,...,...,...,...,...
1212,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,"['The firm accepts people of all race, gender ...",Mid-levelPalo Alto,['2018'],the firm accepts people of all race gender ...
1213,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,['Best: The gender ratio is fairly 1:1 and eve...,San Jose,['2018'],best the gender ratio is fairly and eve...
1214,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,"['new class of ~50+, noticeably male-dominant'...",Palo Alto,['2018'],new class of noticeably male dominant ...
1215,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,['We have very serious issues with diversity a...,Palo Alto,['2018'],we have very serious issues with diversity a...


In [42]:
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1))
text_counts = cv.fit_transform(df["clean_review"])
pd.DataFrame(text_counts)

,0
0,"(0, 2774)\t1\n (0, 1019)\t1\n (0, 491)\t1\..."
1,"(0, 1177)\t1\n (0, 2009)\t1\n (0, 208)\t1\..."
2,"(0, 2302)\t1\n (0, 1111)\t2\n (0, 2364)\t2..."
3,"(0, 1621)\t1\n (0, 2749)\t1\n (0, 376)\t1\..."
4,"(0, 2151)\t1\n (0, 819)\t1\n (0, 2838)\t1\..."
...,...
1212,"(0, 1111)\t1\n (0, 258)\t1\n (0, 1024)\t1\..."
1213,"(0, 1111)\t1\n (0, 2558)\t1\n (0, 2151)\t1..."
1214,"(0, 1573)\t2\n (0, 1771)\t1\n (0, 1716)\t3..."
1215,"(0, 1024)\t1\n (0, 768)\t1\n (0, 1648)\t1\..."
